# 610 WorkFlow Gerencial - Nicolas Horn (FINAL)

**Hiperparametros optimizados (BO 250 iters):**
- num_leaves = 289
- min_data_in_leaf = 279
- max_depth = 10
- lambda_l1 = 0.529
- lambda_l2 = 3.062
- min_gain_to_split = 0.070
- num_iterations = 463

**Feature Engineering:**
- Catastrophe Analysis (13 vars -> NA en 202006)
- Data Drifting por IPC (deflactacion)
- lags (1,2,3) + deltas (1,2,3) + trends (3,6)

**Exclusion de variables:** numero_de_cliente, foto_mes

**Validacion:** 202107 (ganancia local)
**Prediccion Kaggle:** 202109
**5 semillas:** 153929, 838969, 922081, 795581, 194609

## 1. Seteo Google Colab (Python3)

In [ ]:
from google.colab import drive
drive.mount('/content/.drive')

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -sf "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json ~/.kaggle 2>/dev/null || true
chmod 600 ~/.kaggle/kaggle.json 2>/dev/null || true

mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
archivo="gerencial_competencia_2025.csv.gz"

if ! test -f /content/buckets/b1/datasets/$archivo; then
  wget $webfiles/$archivo -O /content/buckets/b1/datasets/$archivo
fi

if ! test -f /content/datasets/$archivo; then
  cp /content/buckets/b1/datasets/$archivo /content/datasets/$archivo
fi

ls -lh /content/datasets/$archivo

## 2. Inicializacion R

**Cambiar Runtime a R**

In [ ]:
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

require("data.table")
if(!require("R.utils")) install.packages("R.utils")
if(!require("lightgbm")) install.packages("lightgbm")
if(!require("yaml")) install.packages("yaml")
require("R.utils")
require("lightgbm")
require("yaml")

cat("Inicio:", format(Sys.time(), "%a %b %d %X %Y"), "\n")

## 3. Configuracion

In [ ]:
PARAM <- list()

# Experimento
PARAM$experimento_base <- 6200
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

# Semillas
PARAM$semillas <- c(153929, 838969, 922081, 795581, 194609)

# Cortes para Kaggle (1000 a 1250, step 50)
PARAM$kaggle_cortes <- seq(1000, 1250, 50)

# Competencia Kaggle
PARAM$kaggle_competencia <- "dm-ey-f-2025-primera"

# Variables a excluir (evitan data leakage)
PARAM$excluir_campos <- c("numero_de_cliente", "foto_mes")

# Meses
PARAM$meses_train <- c(202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
                       202101, 202102, 202103, 202104, 202105, 202106)
PARAM$mes_validacion <- 202107
PARAM$mes_kaggle <- 202109

# Hiperparametros optimizados (BO 250 iters)
PARAM$lgbm <- list(
  objective = "binary",
  metric = "auc",
  first_metric_only = TRUE,
  boost_from_average = TRUE,
  feature_pre_filter = FALSE,
  verbosity = -100,
  force_row_wise = TRUE,
  max_bin = 31,
  learning_rate = 0.03,
  feature_fraction = 0.8,
  bagging_fraction = 0.8,
  bagging_freq = 1,
  num_leaves = 289,
  min_data_in_leaf = 279,
  max_depth = 10,
  lambda_l1 = 0.5291058,
  lambda_l2 = 3.061922,
  min_gain_to_split = 0.07016705,
  num_iterations = 463
)

# IPC para deflactacion
PARAM$ipc <- data.table(
  foto_mes = c(202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105, 202106, 202107, 202109),
  IPC = c(1.2118694724, 1.1881073259, 1.1693969743, 1.1375456949, 1.1065619600,
          1.0681100000, 1.0370000000, 1.0000000000, 0.9680542110, 0.9344152616,
          0.8882274350, 0.8532444140, 0.8251880213, 0.8003763543, 0.7763107219,
          0.7540000000)
)

# Resultados
resultados <- list()

cat("Configuracion:\n")
cat("  Semillas:", length(PARAM$semillas), "\n")
cat("  Cortes Kaggle:", paste(PARAM$kaggle_cortes, collapse=", "), "\n")
cat("  Competencia:", PARAM$kaggle_competencia, "\n")
cat("  Validacion:", PARAM$mes_validacion, "\n")
cat("  Kaggle:", PARAM$mes_kaggle, "\n")

## 4. Funciones Auxiliares

In [ ]:
# Funcion para calcular tendencia (slope)
calc_slope <- function(y) {
  n <- length(y)
  valid <- !is.na(y)
  n_valid <- sum(valid)
  if (n_valid < 2) return(NA_real_)
  x <- 1:n
  x_valid <- x[valid]
  y_valid <- y[valid]
  sum_x <- sum(x_valid)
  sum_y <- sum(y_valid)
  sum_xy <- sum(x_valid * y_valid)
  sum_x2 <- sum(x_valid^2)
  denom <- n_valid * sum_x2 - sum_x^2
  if (denom == 0) return(NA_real_)
  (n_valid * sum_xy - sum_x * sum_y) / denom
}

# Funcion para aplicar Feature Engineering
aplicar_fe <- function(dataset) {
  
  # Catastrophe Analysis (13 variables en 202006)
  dataset[foto_mes==202006, c("internet", "mrentabilidad", "mrentabilidad_annual",
    "mcomisiones", "mactivos_margen", "mpasivos_margen", "mcuentas_saldo",
    "ctarjeta_visa_transacciones", "mtarjeta_visa_consumo", "mtarjeta_master_consumo",
    "ccallcenter_transacciones", "chomebanking_transacciones",
    "ctarjeta_master_transacciones") := NA]
  
  # Data Drifting - IPC
  campos_m <- colnames(dataset)[colnames(dataset) %like% "^m"]
  dataset[PARAM$ipc, on = "foto_mes", (campos_m) := .SD * i.IPC, .SDcols = campos_m]
  
  # FE Intra-mes
  dataset[, kmes := foto_mes %% 100]
  if(all(c("mpayroll", "cliente_edad") %in% colnames(dataset)))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]
  
  # FE Historico
  setorder(dataset, numero_de_cliente, foto_mes)
  cols <- setdiff(colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria"))
  
  # Lags 1, 2, 3
  for (i in 1:3) {
    dataset[, paste0(cols, "_lag", i) := shift(.SD, i, NA, "lag"), 
            by = numero_de_cliente, .SDcols = cols]
  }
  
  # Deltas 1, 2, 3
  for (vcol in cols) {
    for (i in 1:3) {
      dataset[, paste0(vcol, "_delta", i) := get(vcol) - get(paste0(vcol, "_lag", i))]
    }
  }
  
  # Trends 3 y 6
  for (col in cols) {
    dataset[, paste0(col, "_trend_3") := frollapply(get(col), 3, calc_slope, align="right"), 
            by = numero_de_cliente]
    dataset[, paste0(col, "_trend_6") := frollapply(get(col), 6, calc_slope, align="right"), 
            by = numero_de_cliente]
  }
  
  return(dataset)
}

## 5. Loop Principal - 5 Semillas

In [ ]:
for (seed_idx in 1:length(PARAM$semillas)) {
  
  cat("\n\n", strrep("=", 50), "\n")
  cat("SEMILLA", seed_idx, "de", length(PARAM$semillas), 
      "(", PARAM$semillas[seed_idx], ")\n")
  cat(strrep("=", 50), "\n\n")
  
  inicio <- Sys.time()
  semilla <- PARAM$semillas[seed_idx]
  experimento <- PARAM$experimento_base + seed_idx - 1
  
  # Crear carpeta
  carpeta <- paste0("/content/buckets/b1/exp/WF", experimento)
  dir.create(carpeta, showWarnings=FALSE, recursive=TRUE)
  dir.create(paste0(carpeta, "/kaggle"), showWarnings=FALSE)
  setwd(carpeta)
  
  # Cargar y procesar dataset
  cat("Cargando dataset...\n")
  dataset <- fread(paste0("/content/datasets/", PARAM$dataset))
  
  cat("Aplicando Feature Engineering...\n")
  dataset <- aplicar_fe(dataset)
  cat("  Columnas:", ncol(dataset), "\n")
  
  # Preparar datos
  dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1, 0)]
  campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01", "azar", PARAM$excluir_campos))
  
  # Dataset de entrenamiento
  set.seed(semilla, kind = "L'Ecuyer-CMRG")
  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$meses_train, campos_buenos, with=FALSE]),
    label = dataset[foto_mes %in% PARAM$meses_train, clase01]
  )
  
  # Entrenar modelo
  cat("Entrenando modelo...\n")
  param <- c(PARAM$lgbm, list(seed = semilla))
  modelo <- lgb.train(data = dtrain, param = param, verbose = -100)
  lgb.save(modelo, "modelo.txt")
  
  # ===== VALIDACION EN JULIO (202107) =====
  cat("\nValidacion en", PARAM$mes_validacion, "...\n")
  df_val <- dataset[foto_mes == PARAM$mes_validacion]
  pred_val <- predict(modelo, data.matrix(df_val[, campos_buenos, with=FALSE]))
  
  tb_val <- df_val[, list(numero_de_cliente, clase_ternaria)]
  tb_val[, prob := pred_val]
  tb_val[, ganancia := ifelse(clase_ternaria == "BAJA+2", 117000, -3000)]
  setorder(tb_val, -prob)
  tb_val[, gan_acum := cumsum(ganancia)]
  tb_val[, gan_suav := frollmean(gan_acum, 400, align="center", na.rm=TRUE)]
  
  gan_max <- max(tb_val$gan_suav, na.rm=TRUE)
  envios_opt <- which.max(tb_val$gan_suav)
  
  cat("  Ganancia max:", formatC(gan_max, format="f", big.mark=",", digits=0), "\n")
  cat("  Envios optimos:", envios_opt, "\n")
  
  # Guardar ganancias por corte
  gan_cortes <- sapply(PARAM$kaggle_cortes, function(c) tb_val[c, gan_acum])
  names(gan_cortes) <- as.character(PARAM$kaggle_cortes)
  
  fwrite(tb_val, "ganancias_julio.txt", sep="\t")
  
  # ===== PREDICCION PARA KAGGLE (202109) =====
  cat("\nPrediccion para Kaggle (", PARAM$mes_kaggle, ")...\n")
  df_kaggle <- dataset[foto_mes == PARAM$mes_kaggle]
  pred_kaggle <- predict(modelo, data.matrix(df_kaggle[, campos_buenos, with=FALSE]))
  
  tb_kaggle <- df_kaggle[, list(numero_de_cliente)]
  tb_kaggle[, prob := pred_kaggle]
  setorder(tb_kaggle, -prob)
  
  # Generar CSVs con cortes definidos + optimo de Julio
  cortes_generar <- sort(unique(c(PARAM$kaggle_cortes, envios_opt)))
  
  cat("  Generando CSVs (cortes:", paste(cortes_generar, collapse=", "), ")\n")
  for (corte in cortes_generar) {
    tb_kaggle[, Predicted := 0L]
    tb_kaggle[1:corte, Predicted := 1L]
    archivo <- paste0("./kaggle/KA", experimento, "_", corte, ".csv")
    fwrite(tb_kaggle[, list(numero_de_cliente, Predicted)], file=archivo, sep=",")
  }
  
  # Guardar resultado
  resultados[[seed_idx]] <- list(
    seed_idx = seed_idx,
    semilla = semilla,
    experimento = experimento,
    ganancia_max = gan_max,
    envios_opt = envios_opt,
    gan_cortes = gan_cortes
  )
  
  # Guardar PARAM
  write_yaml(list(PARAM = PARAM, resultado = resultados[[seed_idx]]), "PARAM.yml")
  
  # Limpiar
  rm(dataset, dtrain, modelo, df_val, df_kaggle, tb_val, tb_kaggle)
  gc(full=TRUE, verbose=FALSE)
  
  cat("\nCompletado en", round(difftime(Sys.time(), inicio, units="mins"), 1), "min\n")
}

cat("\n", strrep("=", 50), "\n")
cat("TODAS LAS SEMILLAS PROCESADAS\n")
cat(strrep("=", 50), "\n")

## 6. Resumen y Recomendacion

In [ ]:
setwd("/content/buckets/b1/exp")

# Crear tabla resumen
tb_resumen <- rbindlist(lapply(resultados, function(r) {
  data.table(
    seed = r$seed_idx,
    semilla = r$semilla,
    exp = r$experimento,
    ganancia = r$ganancia_max,
    envios_opt = r$envios_opt
  )
}))

# Agregar ganancias por corte
for (corte in PARAM$kaggle_cortes) {
  tb_resumen[, paste0("g_", corte) := sapply(resultados, function(r) r$gan_cortes[as.character(corte)])]
}

cat("\n", strrep("=", 60), "\n")
cat("RESUMEN FINAL\n")
cat(strrep("=", 60), "\n\n")
print(tb_resumen[, .(seed, semilla, ganancia, envios_opt)])

cat("\nESTADISTICAS JULIO (validacion local):\n")
cat("  Ganancia promedio:", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("  Ganancia maxima:", formatC(max(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("  Envios optimos promedio:", round(mean(tb_resumen$envios_opt)), "\n")

cat("\nGANANCIA PROMEDIO POR CORTE (Julio):\n")
for (corte in PARAM$kaggle_cortes) {
  col <- paste0("g_", corte)
  cat(sprintf("  Corte %4d: %s\n", corte, formatC(mean(tb_resumen[[col]]), format="f", big.mark=",", digits=0)))
}

# Guardar resumen
fwrite(tb_resumen, paste0("resumen_exp", PARAM$experimento_base, ".txt"), sep="\t")
saveRDS(resultados, paste0("resultados_exp", PARAM$experimento_base, ".rds"))

In [ ]:
# RECOMENDACION DE CORTES PARA KAGGLE

cat("\n")
cat(strrep("=", 60), "\n")
cat("RECOMENDACION DE CORTES PARA KAGGLE\n")
cat(strrep("=", 60), "\n\n")

envios_prom <- round(mean(tb_resumen$envios_opt))
cat("Envios optimos promedio (Julio):", envios_prom, "\n\n")

# Analizar cortes
analisis <- data.table(
  corte = PARAM$kaggle_cortes,
  gan_prom = sapply(PARAM$kaggle_cortes, function(c) mean(tb_resumen[[paste0("g_", c)]])),
  gan_min = sapply(PARAM$kaggle_cortes, function(c) min(tb_resumen[[paste0("g_", c)]])),
  gan_max = sapply(PARAM$kaggle_cortes, function(c) max(tb_resumen[[paste0("g_", c)]])),
  varianza = sapply(PARAM$kaggle_cortes, function(c) sd(tb_resumen[[paste0("g_", c)]]))
)
analisis[, dist_opt := abs(corte - envios_prom)]
analisis[, score := rank(-gan_prom) + rank(varianza) + rank(dist_opt)]
setorder(analisis, score)

cat("Ranking de cortes:\n")
for (i in 1:nrow(analisis)) {
  r <- analisis[i]
  stars <- paste(rep("*", nrow(analisis) - i + 1), collapse="")
  cat(sprintf("  %s Corte %4d: gan_prom=%s\n", 
      stars, r$corte, formatC(r$gan_prom, format="f", big.mark=",", digits=0)))
}

cat("\n")
cat(strrep("-", 40), "\n")
cat("RECOMENDACION:\n")
cat("  1ra opcion: Corte", analisis[1, corte], "\n")
cat("  2da opcion: Corte", analisis[2, corte], "\n")
cat("  Corte optimo Julio:", envios_prom, "(tambien generado)\n")
cat(strrep("-", 40), "\n")

cat("\nArchivos listos en: /content/buckets/b1/exp/WF", PARAM$experimento_base, "/kaggle/\n", sep="")

## 7. Envio Automatico a Kaggle

In [ ]:
# Funcion para enviar a Kaggle
enviar_kaggle <- function(archivo_csv, mensaje = NULL) {
  if (is.null(mensaje)) {
    mensaje <- basename(archivo_csv)
  }
  cmd <- sprintf('kaggle competitions submit -c %s -f "%s" -m "%s"',
                 PARAM$kaggle_competencia, archivo_csv, mensaje)
  cat("Enviando:", basename(archivo_csv), "\n")
  system(cmd)
  Sys.sleep(2)  # Esperar entre envios para no saturar la API
}

# Enviar todos los CSVs generados (todas las semillas, todos los cortes)
cat("\n")
cat(strrep("=", 60), "\n")
cat("ENVIANDO A KAGGLE\n")
cat("Competencia:", PARAM$kaggle_competencia, "\n")
cat(strrep("=", 60), "\n\n")

envios_total <- 0

for (seed_idx in 1:length(PARAM$semillas)) {
  experimento <- PARAM$experimento_base + seed_idx - 1
  carpeta_kaggle <- paste0("/content/buckets/b1/exp/WF", experimento, "/kaggle")
  
  # Listar todos los CSVs de esta semilla
  archivos <- list.files(carpeta_kaggle, pattern = "^KA.*\\.csv$", full.names = TRUE)
  
  cat("\nSemilla", seed_idx, "(exp", experimento, "):", length(archivos), "archivos\n")
  
  for (archivo in archivos) {
    # Extraer corte del nombre del archivo
    corte <- gsub(".*_(\\d+)\\.csv$", "\\1", basename(archivo))
    mensaje <- paste0("WF", experimento, "_seed", PARAM$semillas[seed_idx], "_corte", corte)
    enviar_kaggle(archivo, mensaje)
    envios_total <- envios_total + 1
  }
}

cat("\n")
cat(strrep("=", 60), "\n")
cat("ENVIOS COMPLETADOS:", envios_total, "archivos\n")
cat(strrep("=", 60), "\n")

In [ ]:
cat("\nFin:", format(Sys.time(), "%a %b %d %X %Y"), "\n")